In [1]:
import chess
import torch

from chess_app.environment import ChessInterface
from re_algoritms.agents import RandomAgent, QAgent
from chess_app.dataset.loader import loaderChessPos, tanstsovVecLoader
import chess.svg
from tqdm import tqdm


In [2]:

def is_checkmate(board, state):
    board.set_fen(state)
    return board.is_checkmate()


In [3]:
from re_algoritms.dqn import GrandMasterNet
from re_algoritms.agents import DQNAgent

board = chess.Board()

e = 0.1
lr = 0.3
gamma = 0.3
n_epochs = 100

policy_net = GrandMasterNet(
    board_vec_dim=68,
    moves_vec_dim=5
)
player = DQNAgent(
    lr=lr, gamma=gamma, model=policy_net
)
loader = tanstsovVecLoader(r'chess_app\dataset\result.json', r'chess_app\dataset\settings.json')
loader.createDf()
dataset, fen_strings = loader.getAllData()

In [4]:
for array, fen_string in zip(dataset.to_numpy(), fen_strings):
    # print(_)
    print(array, fen_string)
    break

[0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] k1K5/8/2R5/8/8/8/8/8 w - - 0 1


In [5]:
# bc_win = 0
# wh_win = 0
# stockfish = ChessInterface(verbose=True, engine_path=r"./chess_app/src/stockfish_15_x64_avx2.exe")
# for epoch in range(n_epochs):
#     for cnt_to_finish in list(loader.games.keys()):
#         for state in tqdm(loader.games[cnt_to_finish]):
#             stockfish.stockfish.set_fen_position(state)
#             # actions = list() # todo: выбранная вероятность
#             rewards = list()     # ...
#
#             move_counter = 1
#             reward = 0
#             black_win = False
#             white_win = False
#             new_state = state
#             b_state = None
#             while not is_checkmate(board, new_state):
#                 stockfish.stockfish.set_fen_position(new_state)
#                 player_move = player.return_move(new_state, stockfish, e)
#                 #print(f'Player move:{player_move}')
#                 stockfish.player_move(player_move)
#                 #new_state = stockfish.stockfish.get_fen_position()
#             # actions.append(new_state) # ?
#                 move_counter += 1
#                 if move_counter > 5:
#                     black_win = True
#                     bc_win += 1
#                     break
#                 # actions.append(new_state)
#
#                 if is_checkmate(board, new_state):
#                     wh_win += 1
#                     white_win = True
#                 else:
#                     machine_move = stockfish.env_move()
#                     new_state = stockfish.get_board_fen()
#                     #print(f'Machine move:{machine_move}')
#                 if is_checkmate(board, new_state) and not white_win:
#                     bc_win += 1
#                     black_win = True
#                 print(actions)
#
#
#
#                 #print('---')
#                 # break
#
#             if black_win is True:
#                 reward = -1
#             else:
#                 reward = 1
#
#             player.update_policy(reward=reward)
#             # break
#         print(f'Win: {wh_win}\t Lose: {bc_win}')

In [6]:
bc_win = 0
wh_win = 0
stockfish = ChessInterface(verbose=True, engine_path=r"./chess_app/src/stockfish_15_x64_avx2.exe")
for epoch in range(n_epochs):
    for array, fen_string in tqdm(zip(dataset.to_numpy(), fen_strings)):
        # init necessary for game
        new_state = fen_string
        new_array = array

        black_win = False
        white_win = False

        stockfish.set_board_fen(new_state)
        all_rewards = list()

        for _ in range(5):
            # player make move
            stockfish.set_board_fen(new_state)
            player_move = player.return_move(
                new_array,
                stockfish.get_top_steps() # todo
            )
            stockfish.player_move(player_move)

            all_rewards.append(0)

            if is_checkmate(board, new_state):
                white_win = True
                break

            # environment make move
            machine_move = stockfish.env_move()
            new_state = stockfish.get_board_fen()

            new_array = loader.extractPos(new_state, -1, -1)
            new_array = new_array.values()[3:]

            if is_checkmate(board, new_state):
                black_win = True
                break
        else:
            black_win = True


        if black_win is True:
            reward = -1
        else:
            reward = 1

        all_rewards.append(reward)
        player.update_policy(all_rewards)

0it [00:00, ?it/s]

3616
3837
3626
3676
3632
torch.Size([1, 68]) torch.Size([1, 5])


0it [00:00, ?it/s]


IndexError: index 3 is out of bounds for dimension 0 with size 1